In [1]:
# !pip install ultralytics

In [2]:
from ultralytics import YOLO
import numpy as np
import cv2
import pandas as pd


In [3]:

class YOLOSegmentation:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def detect(self, img):
        height, width, channels = img.shape

        results = self.model.predict(source=img.copy(), save=False, save_txt=False)
        result = results[0]
        segmentation_contours_idx = []
        for seg in result.masks.xyn:
            seg[:, 0] *= width
            seg[:, 1] *= height
            segment = np.array(seg, dtype=np.int32)
            segmentation_contours_idx.append(segment)

        bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        scores = np.array(result.boxes.conf.cpu(), dtype="float").round(2)
        return bboxes, class_ids, segmentation_contours_idx, scores


In [4]:
PATH_TO_READ_FROM = "AUDI_100_00002.jpg"

In [5]:
img = cv2.imread(PATH_TO_READ_FROM)
img = cv2.resize(img, None, fx=0.5, fy=0.5)


ys = YOLOSegmentation("yolov8m-seg.pt")

bboxes, classes, segmentations, scores = ys.detect(img)


0: 352x640 1 car, 1630.2ms
Speed: 15.5ms preprocess, 1630.2ms inference, 15.6ms postprocess per image at shape (1, 3, 352, 640)


In [6]:
PATH_TO_WRITE_TO = "AUDI_processed_1.jpg"

In [7]:
# Все за пределами сегментированного изображения станет черным
points = np.array(segmentations)

mask = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)
cropped_img = cv2.bitwise_and(img, img, mask=mask)


cv2.imwrite((PATH_TO_WRITE_TO), cropped_img)

True

In [8]:
image3 = cv2.imread((PATH_TO_WRITE_TO))
blurred_image = cv2.GaussianBlur(image3.copy(),(5,5),0)

edges = cv2.Canny(blurred_image, 100, 160)

cv2.imwrite((PATH_TO_WRITE_TO), edges)



True